# Laserchicken experiment
This document shows how Laserchicken can be used to calculate features that describe a point cloud.

In [81]:
from laserchicken import load
from laserchicken import compute_features
import requests
from laserchicken.normalize import normalize
from laserchicken.filter import select_polygon

## Load a las file
input_data_set_url = "https://github.com/QCDIS/lifewatch-notebooks/raw/main/eEcolidar/testdata/AHN3.las"
r = requests.get(input_data_set_url)
with open("/tmp/input_data_set.laz", 'wb') as f:
    f.write(r.content)

point_cloud = load('/tmp/input_data_set.laz')


## Normalize data
# Adds the attribute 'normaiized_height' to the point cloud. Note that data is added to the given point cloud  in place 
point_cloud_normalized = normalize(point_cloud)
point_cloud_normalized

## Filter points inside a polygon
# We filter points within a polygon


polygon = "POLYGON(( 131963.984125 549718.375000," + \
                   " 132000.000125 549718.375000," + \
                   " 132000.000125 549797.063000," + \
                   " 131963.984125 549797.063000," + \
                   " 131963.984125 549718.375000))"

point_cloud_selected = select_polygon(point_cloud, polygon)

## Filter points by attribute threshold

from laserchicken.filter import select_above, select_below
points_below_1_meter = select_below(point_cloud_selected, 'normalized_height', 1)
points_above_1_meter = select_above(point_cloud_selected, 'normalized_height', 1)

export(points_below_1_meter, 'points_below_1_meter.ply')
export(points_above_1_meter, 'points_above_1_meter.ply')


import aioeasywebdav
import asyncio
webdav_host = 'http://172.17.0.3'
username='alice'
password='secret1234'

loop = asyncio.get_event_loop()
webdav = aioeasywebdav.connect(webdav_host, username=username, password=password,protocol='http')
await webdav.upload('points_above_1_meter.ply', 'points_above_1_meter.ply')
await webdav.upload('points_below_1_meter.ply', 'points_below_1_meter.ply')

# Compute neighbors
# Computer neighbors for every point in the point cloud. This means that the target point cloud should contain the same points as the environment point cloud. The target point cloud contains the point at which the neighbors should be calculated.
# The code below finds for every target point, the neighbors within a Spherical neighborhood with radius of 5 meters.

from laserchicken import compute_neighborhoods
from laserchicken import build_volume
targets = point_cloud_selected
volume = build_volume('sphere', radius=5)
neighborhoods = compute_neighborhoods(point_cloud_selected, targets, volume)

## Calculate features

from laserchicken import compute_features
compute_features(point_cloud_selected, neighborhoods, targets, ['std_z','mean_z','slope'], volume)


## Calculate parameterized features
# We add 4 parameterized features.
from laserchicken import register_new_feature_extractor
from laserchicken.feature_extractor.band_ratio_feature_extractor import BandRatioFeatureExtractor
register_new_feature_extractor(BandRatioFeatureExtractor(None,1,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(1,2,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(2,None,data_key='normalized_height'))
register_new_feature_extractor(BandRatioFeatureExtractor(None,0,data_key='z'))


# Print a list of all available feature names and notice the parameterized features we just registered, 'band_ratio_1<z<2'.

from laserchicken.feature_extractor.feature_extraction import list_feature_names
sorted(list_feature_names())

## Recalculate neighbors and calculate the parameterized feature
# Neighbors is a generator and can only be iterated once. We therefore recalculate them here. If we would want 
# to do multiple calculations without recalculating the neighbors, we could copy the neighbors to a list.
# This is not done by default because neighbors can quickly grow quite large so that available RAM unnecessarily 
# becomes the bottle neck.

## Write result to ply file
from laserchicken import export
export(point_cloud, 'point_cloud.ply')
export(point_cloud_selected, 'point_cloud_selected.ply')



import aioeasywebdav
import asyncio
webdav_host = 'http://172.17.0.3'
username='alice'
password='secret1234'

loop = asyncio.get_event_loop()
webdav = aioeasywebdav.connect(webdav_host, username=username, password=password,protocol='http')
await webdav.upload('point_cloud.ply', 'point_cloud.ply')
await webdav.upload('point_cloud_selected.ply', 'point_cloud_selected.ply')



Cylinder size in Bytes: 1262794583.0369534
Memory size in Bytes: 7779377152
Start tree creation
Done with env tree creation
Done with target tree creation
Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']"Extracting feature(s) "['eigenv_1', 'eigenv_2', 'eigenv_3', 'normal_vector_1', 'normal_vector_2', 'normal_vector_3', 'slope']" took 0.23 seconds
Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']"Extracting feature(s) "['mean_z', 'std_z', 'coeff_var_z']" took 0.25 seconds
The following unrequested features were calculated as a side effect, but will not be returned: ['normal_vector_3', 'normal_vector_2', 'normal_vector_1', 'eigenv_3', 'eigenv_2', 'eigenv_1', 'coeff_var_z']


## Visulize 

In [82]:
import numpy as np
import open3d as o3d

import aioeasywebdav
import asyncio
webdav_host = 'http://172.17.0.3'
username='alice'
password='secret1234'

loop = asyncio.get_event_loop()
webdav = aioeasywebdav.connect(webdav_host, username=username, password=password,protocol='http')
res = await webdav.download('/points_above_1_meter.ply', 'download_points_above_1_meter.ply')
res = await webdav.download('/point_cloud.ply', 'download_point_cloud.ply')
res = await webdav.download('/point_cloud_selected.ply', 'download_point_cloud_selected.ply')


pcd = o3d.io.read_point_cloud("download_point_cloud.ply")
pcd
np.asarray(pcd.points)
o3d.visualization.draw_geometries([pcd])


pcd = o3d.io.read_point_cloud("download_point_cloud_selected.ply")
pcd
np.asarray(pcd.points)
o3d.visualization.draw_geometries([pcd])
